<a href="https://colab.research.google.com/github/grillinr/evolutionary-computing/blob/main/final/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and seed for easier checking

In [17]:
import random
import os
import argparse

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


SEED = 5173
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")
print(device)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

cuda


# Define helper functions

In [24]:
def prepare_data_with_scaler(data, scaler=None, fit=False):
    data = data.dropna()
    X = data.drop(columns=["id", "record", "type"]).values.astype(np.float32)
    y = data["type"].astype("category").cat.codes.values

    if fit:
        X = scaler.fit_transform(X)
    else:
        X = scaler.transform(X)

    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)


def evaluate(model, X, y, criterion):
    model.eval()
    with torch.no_grad():
        logits = model(X)
        loss = criterion(logits, y)
        y_pred = logits.argmax(dim=1).cpu().numpy()

    y_true = y.cpu().numpy()
    return {
        "loss": loss.item(),
        "accuracy": accuracy_score(y_true, y_pred),
        "f_beta_macro": fbeta_score(y_true, y_pred, average="macro", beta=2, zero_division=0)
    }

# Create Model Architecture (DNN)

In [19]:
class DNN(nn.Module):
    def __init__(self, input_size=32, hidden=(32, 16, 8), num_classes=5, dropout_rate=0.5):
        super().__init__()
        layers = []
        input_dim = input_size

        for h in hidden:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout_rate))
            input_dim = h

        layers.append(nn.Linear(input_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Main Training loop

In [25]:
# Load data
dataset = pd.read_csv("/content/train.csv")
train_dataset, val_dataset = train_test_split(dataset, train_size=0.7, random_state=SEED)
scaler = StandardScaler()
X_train, y_train = prepare_data_with_scaler(train_dataset, scaler, fit=True)
X_val, y_val = prepare_data_with_scaler(val_dataset, scaler, fit=False)

X_train, y_train = X_train.to(device), y_train.to(device)
X_val, y_val = X_val.to(device), y_val.to(device)

In [26]:
# Configuration
lr = 1e-3
epochs = 1000
hidden = (32, 16, 8)
dropout_rate = 0.5
patience = 100

# Create model
model = DNN(hidden=hidden, dropout_rate=dropout_rate).to(device)

class_counts = train_dataset['type'].value_counts()
weights = 1.0 / class_counts.values
weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [28]:
# Training loop with early stopping
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    train_metrics = evaluate(model, X_train, y_train, criterion)
    val_metrics = evaluate(model, X_val, y_val, criterion)

    if epoch % 10 == 0:
      print(
          f"Epoch {epoch}/{epochs} | "
          f"train_loss={train_loss:.4f} train_acc={train_metrics['accuracy']:.4f} "
          f"train_f1={train_metrics['f_beta_macro']:.4f} "
          f"val_loss={val_metrics['loss']:.4f} val_acc={val_metrics['accuracy']:.4f} "
          f"val_f1={val_metrics['f_beta_macro']:.4f} "
      )

    # Early stopping check
    if val_metrics['loss'] < best_val_loss:
        best_val_loss = val_metrics['loss']
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

Epoch 10/1000 | train_loss=1.3987 train_acc=0.0674 train_f1=0.0546 val_loss=1.3922 val_acc=0.0675 val_f1=0.0557 
Epoch 20/1000 | train_loss=1.2662 train_acc=0.0677 train_f1=0.0533 val_loss=1.2205 val_acc=0.0678 val_f1=0.0533 
Epoch 30/1000 | train_loss=1.1273 train_acc=0.0677 train_f1=0.0533 val_loss=1.0182 val_acc=0.0678 val_f1=0.0533 
Epoch 40/1000 | train_loss=1.0116 train_acc=0.0677 train_f1=0.0533 val_loss=0.8405 val_acc=0.0678 val_f1=0.0533 
Epoch 50/1000 | train_loss=0.9290 train_acc=0.0677 train_f1=0.0533 val_loss=0.7303 val_acc=0.0678 val_f1=0.0533 
Epoch 60/1000 | train_loss=0.8862 train_acc=0.0677 train_f1=0.0533 val_loss=0.6775 val_acc=0.0678 val_f1=0.0533 
Epoch 70/1000 | train_loss=0.8514 train_acc=0.0677 train_f1=0.0533 val_loss=0.6510 val_acc=0.0678 val_f1=0.0533 
Epoch 80/1000 | train_loss=0.8295 train_acc=0.0677 train_f1=0.0533 val_loss=0.6332 val_acc=0.0678 val_f1=0.0533 
Epoch 90/1000 | train_loss=0.8015 train_acc=0.0677 train_f1=0.0533 val_loss=0.6199 val_acc=0.067

# Test output

In [ ]:
X_test, y_test = prepare_data(test_dataset, device)

# Get predictions
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    predictions = probs.argmax(axis=1)

y_true = y_test.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_true, predictions)
f_beta = fbeta_score(y_true, predictions, average="macro", beta=2)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F-Beta (macro): {f_beta:.4f}")